#

# DO NOT RUN ALL CELLS (FILE DELETION)
# "####################################"
# Import all the libraries that we need

In [ ]:
import csv
import datetime
import fnmatch
import glob
import itertools
from itertools import groupby
import math
import matplotlib.cbook
import matplotlib.gridspec as gridspec
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import pandas as pd
import pickle
import platform
from pylab import rcParams
import re
import scipy
from scipy import stats
from scipy.signal import find_peaks
import statistics
import warnings
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)

# Define #ROOT

In [ ]:
###Set the notebook screen ratio for jupyter
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:85% !important; }</style>"))

notebookPath=os.path.dirname(os.path.realpath("__file__"))
#pathToProcessingNotebooks=os.path.join(os.path.split(notebookPath)[0],"load_preprocess_rat")
#os.chdir(pathToProcessingNotebooks)   
#%run loadRat_documentationmod.ipynb
#%run plotRat_documentation_1_GeneralBehaviormod.ipynb

#### DEFINE ROOT
#folder with data 
if platform.system()=='Linux':
    root="/home/david/Desktop/DATARUN ANALYSIS/test0715"
    savePath="/home/david/Desktop/Save"
else:
    #mac = 'Darwin'
    root="/Users/tom/Desktop/DATARUN/test0820" #current : test0826
    savePath="/Users/tom/Desktop"#os.path.join(root,"ALLRATS","Fig")
print("Root: ", root)

retval = os.getcwd()
print("Current working directory: %s" % retval)#print("Notebook Path: ", notebookPath)
print("Save Path: ", savePath)   
print("Path to data is: %s"%root)
print("")
print("importing done")

# Define utility functions
# + details
# columns in file from LV

In [ ]:
#conversion functions
#check if used 
def inch2cm(value):
    return value/2.54
def cm2inch(value):
    return value*2.54
def px2cm(value):
    return value/13.5

#function to split lists --> used to split the raw X position array into smaller arrays (runs and stays). Later in the code we modify the array and change some values to 0, which will be used as cutting points.
def split_a_list_at_zeros(List):
    return [list(g) for k, g in groupby(List, key=lambda x:x!=0) if k]

#function to open and read from the .position files using pandas, specify the path of the file to open, the column that you want to extract from, and the extension of the file
def read_csv_pandas(path,Col=None,header=None):
    #verify that the file exists
    if not os.path.exists(path):
        print("No file %s"%path)
        return []
    try:
        #open the file
        csvData=pd.read_csv(path,header=header,delim_whitespace=True)
    except ValueError:
        print("%s not valid (usually empty)"%path)
        return []
        #verify that the column that we specified is not empty, and return the values
    if Col is not None:
        return csvData.values[:, Col[0]]
    else:
        return csvData

# function to cut the 30min session (position plus time) into N bins of equal lenght, specify the data to cut (position), data template(we make the bins of a certain lenght so we use time), bot is where we start (usually 0, but can start later if we don't want the start), end is the end of what we want (usually the end of the session), binNumber is the number of bin that we want.
def bin_session(data_to_cut, data_template,  bot, end, binNumber):
    output={}
    for count in range(0, binNumber):
        start_of_bin = count*end/binNumber
        end_of_bin = count*end/binNumber+end/binNumber
        output[count] = data_to_cut[animal, session][np.logical_and(data_template[animal, session] >= start_of_bin, data_template[animal, session] < end_of_bin)]
        #print(boot, toop, output, len(output))    
    return output

#function to read the parameters for each rat for the session in the behav.param file. Specify the name of the parameter that you want to get from the file and optionally the value type that you want. File path is not an option, maybe change that.
def read_params(paramName, valueType=str):
    #define path of the file
    behav = root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".behav_param"
    #check if it exists
    if not os.path.exists(behav):
        print("No file %s" % behav)  
    #check if it is not empty
    #if os.stat(behav).st_size == 0:
        #print("File empty %s" % behav)
    #open the file
    with open(behav, "r") as f:
        #scan the file, if the name of the parameter is here, get the value 
        #scan the file
        for line in f:
            #if you look for a string do that
            if valueType is str:
                #find the line in which you have your parameter
                if paramName in line:
                    #get the last value of the line [-1], values are separated with _blanks_ with the .split() function
                    return int(line.split()[-1])  
                    #same if you want a float value, just convert the value
            if valueType is float: 
                if paramName in line:
                    return float(line.split()[-1])
            #can add other value type if needed (boolean, etc. )

# Load Data
### Data is loaded in one dictionary per variable (time, position, etc.) and have [animal, session] as keys, so we have a set of data for each session of each rat
### This cell loads all the behavioral data from the files created with labview

In [ ]:
# initialise arrays and dicts

rawTime={}
rawPositionX={}
rawPositionY={}
extractPositionX = {}
extractPositionY = {}
extractTime = {}
extractLickLeft = {}
extractLickRight = {}
rawLickLeft1X = {}
rawLickRight1X = {}
rawLickLeft1Y = {}
rawLickRight1Y = {}


#create the list of the animals and sessions
animalindex=1
animalList = [os.path.basename(path) for path in sorted(glob.glob(root+"/Rat*"))]
for animal in animalList:
    print("Animal %s"%animal)
    print("%s out of %s" %(animalindex,len(animalList)))
    print("**********")
    animalindex += 1
    #Get the list of all session
    sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]
    sessionList=sorted(sessionList)
    nbSession=len(sessionList)
    
    #loop through animals and sessions
    for index, session in enumerate(sessionList):
        print("Session %s/%s: %s"%(index+1,nbSession,session), "['" +animal+ "\',\'" +session + "\']")
        acquisitionCutOff = read_params("sessionDuration")
 
        #load data for each session
        #data loaded : time array, position of the animal X and Y axis, Licks to the left and to the right
        extractTime[animal, session]      = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position"),Col=[3])#old format = 5
        extractPositionX[animal, session] = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position"),Col=[4])#old format = 6
        extractPositionY[animal, session] = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position"),Col=[5])
        extractLickLeft[animal, session]  = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position"),Col=[6])
        extractLickRight[animal, session] = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position"),Col=[7])
        #directly cut the data at the end of the session  #explain
        rawTime[animal, session]       = extractTime[animal, session][extractTime[animal, session] <= acquisitionCutOff]
        rawPositionX[animal, session]  = extractPositionX[animal, session][extractTime[animal, session] <= acquisitionCutOff]
        rawPositionY[animal, session]  = extractPositionY[animal, session][extractTime[animal, session] <= acquisitionCutOff]
        rawLickLeft1X[animal, session]  = extractLickLeft[animal, session][extractTime[animal, session] <= acquisitionCutOff]
        rawLickLeft1Y[animal, session]  = extractLickLeft[animal, session][extractTime[animal, session] <= acquisitionCutOff]# not needed, to check
        rawLickRight1X[animal, session] = extractLickRight[animal, session][extractTime[animal, session] <= acquisitionCutOff]
        rawLickRight1Y[animal, session] = extractLickRight[animal, session][extractTime[animal, session] <= acquisitionCutOff]#
        
        print()   
#clear_output()

In [ ]:
extractTime

In [ ]:
# lick data as array of TRUE, FALSE values
# only needed to plot the data 
# change the false values to None
# change true to the position of the rat at this instant
for animal in animalList:
    sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]
    for session in sessionList: 
        #when true use rawpositionX, when false use None
        rawLickLeft1X[animal, session][rawLickLeft1X[animal, session]==True] = rawPositionX[animal, session][rawLickLeft1X[animal, session]==True]
        rawLickLeft1X[animal, session][rawLickLeft1X[animal, session]==False] = 0    
        rawLickRight1X[animal, session][rawLickRight1X[animal, session]==True] = rawPositionX[animal, session][rawLickRight1X[animal, session]==True]
        rawLickRight1X[animal, session][rawLickRight1X[animal, session]==False] = 0

        #same for Y position
        rawLickLeft1Y[animal, session][rawLickLeft1Y[animal, session]==False] = 0
        rawLickLeft1Y[animal, session][rawLickLeft1Y[animal, session]==True] = rawPositionY[animal, session][rawLickLeft1Y[animal, session]==True]
        rawLickRight1Y[animal, session][rawLickRight1Y[animal, session]==False] = 0
        rawLickRight1Y[animal, session][rawLickRight1Y[animal, session]==True] = rawPositionY[animal, session][rawLickRight1Y[animal, session]==True]

In [ ]:
params = {}
params[animal, session] = []
for animal in animalList:
    sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]
    for session in sessionList: 
        #create a dictionary and read the params wor each rat
        params[animal, session] = {
        "sessionDuration" : read_params("sessionDuration"),
        "acqPer" : read_params("acqPer"),
        "waterLeft" : round((read_params("waterLeft", valueType = float) - read_params("cupWeight", valueType = float))/10*1000, 2),
        "waterRight" : round((read_params("waterRight", valueType = float) - read_params("cupWeight", valueType = float))/10*1000, 2),  
        "treadmillSize" : read_params("treadmillSize"), #remove ?     
        "boundaryLeft" : read_params("boundaryLeft"),  #remove
        "boundaryRight" : read_params("boundaryRight"), #removed
        "middle"       : (read_params("boundaryLeft") + read_params("boundaryRight"))/2,  #removed ?
        "weight" : read_params("ratWeight"), 
        "lastWeightadlib" : read_params("ratWeightadlib"),
        "lastDayadlib" : read_params("lastDayadlib")
        }

        lastDayadlib = str(datetime.datetime.strptime(str(read_params("lastDayadlib")), "%Y%m%d").date())
        match = re.search(r'\d{4}_\d{2}_\d{2}', session)
        experimentDay = str(datetime.datetime.strptime(match.group(), '%Y_%m_%d').date())
        lastDayadlib = datetime.date(int(lastDayadlib[0:4]), int(lastDayadlib[5:7]), int(lastDayadlib[8:10]))
        experimentDay = datetime.date(int(experimentDay[0:4]), int(experimentDay[5:7]), int(experimentDay[8:10]))
        daysSinceadlib = experimentDay-lastDayadlib
        params[animal, session]["daysSinceadLib"] = daysSinceadlib.days
        #labview ok 27/07/2020 before nOk #format behavparam ?
        #catchup manual up to 27/07
        params[animal, session]["boundaries"] = []

In [ ]:
print(params)

In [ ]:
# This function plots the base trajectory of the rat. Parameters are time : time data, position : X position data, lickL/R, lick data, maxminstep for x and y axis, color and marker of the plot, width of the axis, and x y labels
def plot_BASEtrajectory(time, position, lickLeft, lickRight, maxminstep, maxminstep2, color = [], marker = [], linewidth = [], xyLabels=["N","Bins"]):   
        plt.plot(time, position, color=color[0], marker=marker[0], linewidth = linewidth[0])
        plt.plot(time, [None if x == 0 else x for x in lickLeft], color=color[1], marker=marker[1], markersize = marker[2])
        plt.plot(time, [None if x == 0 else x for x in lickRight], color=color[1], marker=marker[1], markersize = marker[2])

        ax = plt.gca()
        #ax.set_xlim(maxminstep[:2])
        #configure ax xy span
        ax.set_xlim(maxminstep[0]-maxminstep[2],maxminstep[1]+maxminstep[2])
        ax.set_ylim(maxminstep2[0]-maxminstep2[2],maxminstep2[1]+maxminstep2[2])
        #configure xy style
        ax.set_xlabel(xyLabels[1],fontsize=12,labelpad=0)
        ax.set_ylabel(xyLabels[0],fontsize=12,labelpad=-1)
        ax.xaxis.set_ticks_position('bottom')
        ax.yaxis.set_ticks_position('left')
        ax.get_xaxis().set_tick_params(direction='out',pad=2)
        ax.get_yaxis().set_tick_params(direction='out',pad=2)
        #remove unnecessary axes
        ax.spines['top'].set_color("none")
        ax.spines['right'].set_color("none")
        return ax

In [ ]:
#print(round(params[animal, session]["waterLeft"], 4), "µL/drop")
#print(round(params[animal, session]["waterRight"], 3), "µL/drop")
#print()
#watL = round(params[animal, session]["waterLeft"], 4)
#watR = round(params[animal, session]["waterRight"], 3)

#watL = 9.3
#watR = 9.6

water={}
water[animal] = {}
limitWater_diff = 5 #in%, if waterRight > waterLeft + X% or waterRight < waterLeft - X% we consider that Left is different from Right, maybe try 5% 

for animal in animalList:
    sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]
    for session in sessionList: 
        watL = round(params[animal, session]["waterLeft"], 4)
        watR = round(params[animal, session]["waterRight"], 3)
        print(round(params[animal, session]["waterLeft"], 4), "µL/drop")
        print(round(params[animal, session]["waterRight"], 3), "µL/drop")
        if watL-(watL*limitWater_diff/100) <= watR <= watL+(watL*limitWater_diff/100):
            #print(watL-(watL*limitWater_diff/100))
            #print(watL+(watL*limitWater_diff/100))
            print(watL, watR)
            water[animal] = ["Same Reward Size", "Same Reward Size", 2, 2]
            print("same L-R")
        elif watL < watR:
            print(watL, watR)
            water[animal] = ["Small Reward", "Big Reward", 1, 5]
            print("bigR")
        elif watL > watR:
            print(watL, watR)
            water[animal] = ["Big Reward", "Small Reward", 5, 1]
            print("bigL")
        else:
            print("error, bypass? Y/N")
        print()

In [ ]:
plt.figure(figsize = [20, 5])
for animal in animalList:
    sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]
    for session in sessionList: 
        plt.figure(figsize = [30, 5])
        plt.suptitle(animal + session, y= 1, fontsize = 24)
        #print(animal, session)
        plot_BASEtrajectory(rawTime[animal, session],rawPositionX[animal, session], rawLickLeft1X[animal, session], rawLickRight1X[animal, session], [0,1800,1],[0,1400,1],  color = ["b", "c"], marker = ["", "o", 5], linewidth = [0.5], xyLabels=["Position (cm)", "Time(s)"])


# merge lick file and position file (old format)
# !!!!! BE CAREFUL FILE DELETION !!!!!
# the rest of the notebook should not work properly, if it works you don't need this part

In [ ]:
rawTime={}
rawPositionX={}
rawPositionY={}
extractPositionX = {}
extractPositionY = {}
extractTime = {}
extractLickLeft = {}
extractLickRight = {}
rawLickLeft1X = {}
rawLickRight1X = {}
rawLickLeft1Y = {}
rawLickRight1Y = {}


state1={}
state2={}
state3={}
time1 ={}
x = {}
y = {}
buffer1 = {}
VL = {}
VR = {}

time2 = {}
lickL= {}
lickR={}
buffer2={}
edited={}

#create the list of the animals and sessions
animalindex=1
animalList = [os.path.basename(path) for path in sorted(glob.glob(root+"/Rat*"))]
for animal in animalList:
    print("Animal %s"%animal)
    print("%s out of %s" %(animalindex,len(animalList)))
    print("**********")
    animalindex += 1
    #Get the list of all session
    sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]
    sessionList=sorted(sessionList)
    nbSession=len(sessionList)
    
    #loop through animals and sessions
    for index, session in enumerate(sessionList):
        print("Session %s/%s: %s"%(index+1,nbSession,session), "['" +animal+ "\',\'" +session + "\']")
        acquisitionCutOff = read_params("sessionDuration")
 
        #load data for each session
        #data loaded : time array, position of the animal X and Y axis, Licks to the left and to the right
        state1[animal, session] = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position"),Col=[0])#old format = 5
        state2[animal, session] = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position"),Col=[1])#old format = 6
        state3[animal, session] = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position"),Col=[2])
        #
        #
        time1[animal, session]  = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position"),Col=[5])
        x[animal, session] = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position"),Col=[6])
        y[animal, session] = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position"),Col=[7])
        #
        buffer1[animal, session]  = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position"),Col=[9])
        VL[animal, session] = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position"),Col=[10])
        VR[animal, session] = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position"),Col=[11])

        ###
        time2[animal, session]  = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".lick"),Col=[3])
        lickL[animal, session] = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".lick"),Col=[4])
        lickR[animal, session] = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".lick"),Col=[5])
        buffer2[animal, session]  = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".lick"),Col=[6])
        edited[animal, session] = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".lick"),Col=[7])
        
        print()   
#clear_output()

In [ ]:
b = {}
dfas = {}
dfasT = {}
for animal in animalList:
    sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]
    for session in sessionList: 
        print(animal)
        b[animal, session] = (state1[animal, session], state2[animal, session], state3[animal, session], time1[animal, session], x[animal, session], y[animal, session], lickL[animal, session], lickR[animal, session], VL[animal, session] ,VR[animal, session], buffer1[animal, session], edited[animal, session])
        dfas[animal, session] = pd.DataFrame(b[animal, session])
        dfasT[animal, session] = dfas[animal, session].transpose()
        dfasT[animal, session].to_csv(path_or_buf = root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position1", sep = " ", header = False, index = False, float_format = '%.3f')

In [ ]:
#VR[animal, session] = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position"),Col=[11])

#time2[animal, session]  = read_csv_pandas((root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".lick"),Col=[3])

for animal in animalList:
    sessionList=[os.path.basename(expPath) for expPath in glob.glob(root+os.sep+animal+os.sep+"Experiments"+os.sep+"Rat*")]
    for session in sessionList:
        if os.path.exists(root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position1"):
            if os.path.exists(root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".lick"):
                os.remove(root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".lick")
            if os.path.exists(root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position"):
                os.remove(root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position")
        if not os.path.exists(root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".lick"):
            if not os.path.exists(root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position"):
                print("no olddata")
                if os.path.exists(root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position1"):
                    os.rename(root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position1", root+os.sep+animal+os.sep+"Experiments"+os.sep + session + os.sep+session+".position")

            print(animal, session)